In [1]:
from transformers import AutoConfig, AutoModelForCausalLM
import os
from accelerate import infer_auto_device_map, init_empty_weights
import utils

utils.reload()

[2023-09-05 22:21:26,970] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
# model_name_or_path = "/data/tongyx361/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9"
# model_path = utils.default_13b_model_path
model_path = "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--WizardLM--WizardMath-70B-V1.0/snapshots/e089c3f9d2ad9d1acb62425aec3f4126f498f4c5"
model_config_path = os.path.join(model_path, "config.json")

In [3]:
model_config = AutoConfig.from_pretrained(model_config_path)
print(model_config)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(model_config)

total_params = sum(dict((p.data_ptr(), p.numel()) for p in model.parameters()).values())
print(f"total_params = {total_params}")

for name, param in model.named_parameters():
    print(name, param.shape)


device_map = infer_auto_device_map(model)
print(device_map)

LlamaConfig {
  "_name_or_path": "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--WizardLM--WizardMath-70B-V1.0/snapshots/e089c3f9d2ad9d1acb62425aec3f4126f498f4c5/config.json",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": false,
  "vocab_size": 32002
}



total_params = 262160384
model.embed_tokens.weight torch.Size([32002, 8192])
model.layers.0.self_attn.q_proj.weight torch.Size([8192, 8192])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 8192])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 8192])
model.layers.0.self_attn.o_proj.weight torch.Size([8192, 8192])
model.layers.0.mlp.gate_proj.weight torch.Size([28672, 8192])
model.layers.0.mlp.up_proj.weight torch.Size([28672, 8192])
model.layers.0.mlp.down_proj.weight torch.Size([8192, 28672])
model.layers.0.input_layernorm.weight torch.Size([8192])
model.layers.0.post_attention_layernorm.weight torch.Size([8192])
model.layers.1.self_attn.q_proj.weight torch.Size([8192, 8192])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 8192])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 8192])
model.layers.1.self_attn.o_proj.weight torch.Size([8192, 8192])
model.layers.1.mlp.gate_proj.weight torch.Size([28672, 8192])
model.layers.1.mlp.up_proj.weight torch.Si

In [16]:
for k, v in model.named_modules():
    print(k, v)

 LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32002, 8192, padding_idx=0)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (up_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (down_proj): Linear(in_features=28672, out_features=8192, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSN

In [4]:
print(model_config)

LlamaConfig {
  "_name_or_path": "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--WizardLM--WizardMath-70B-V1.0/snapshots/e089c3f9d2ad9d1acb62425aec3f4126f498f4c5/config.json",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": false,
  "vocab_size": 32002
}



In [7]:
h = hidden_size = model_config.hidden_size
# intermediate_size = model_config.intermediate_size
intermediate_size = 4 * h
V = vocab_size = model_config.vocab_size
l = num_hidden_layers = model_config.num_hidden_layers
# assert (
#     model_config.num_attention_heads == model_config.num_key_value_heads
# ), f"num_attention_heads ({model_config.num_attention_heads}) != num_key_value_heads ({model_config.num_key_value_heads})"

print(f"model_config.num_attention_heads = {model_config.num_attention_heads}")
print(f"model_config.num_key_value_heads = {model_config.num_key_value_heads}")

a = model_config.num_attention_heads

num_gpus = 4

b = batch_size_per_gpu = 2
# s = max_seq_len = 4096
s = max_seq_len = 1024

# b = batch_size_per_gpu = 20
# b = batch_size_per_gpu = 3
# s = max_seq_len = 2048

# num_gpus = 8

model_config.num_attention_heads = 64
model_config.num_key_value_heads = 8


In [9]:
num_params_per_self_attention = 4 * (h**2 + h)
num_params_per_ffn = h * intermediate_size * 2 + intermediate_size + h
num_params_per_layer_norm = 2 * h
num_params_per_transformer_layer = (
    num_params_per_self_attention + num_params_per_ffn + 2 * num_params_per_layer_norm
)

num_params_token_embedding_matrix = vocab_size * hidden_size

num_params_transformer_model = (
    num_params_per_transformer_layer * num_hidden_layers
    + num_params_token_embedding_matrix
)


print("Number of parameters per self-attention layer:", num_params_per_self_attention)
print("Number of parameters per feed-forward network:", num_params_per_ffn)
print("Number of parameters per transformer layer:", num_params_per_transformer_layer)
print(
    "Number of parameters in the token embedding matrix:",
    num_params_token_embedding_matrix,
)
print("Number of parameters in the transformer model:", num_params_transformer_model)

Number of parameters per self-attention layer: 268468224
Number of parameters per feed-forward network: 536911872
Number of parameters per transformer layer: 805412864
Number of parameters in the token embedding matrix: 262160384
Number of parameters in the transformer model: 64695189504


In [10]:
print(4096**2)
print(12 * 4096**2)
print(12 * 32 * 4096**2)

16777216
201326592
6442450944


In [11]:
# mixed precision
mem_params_mixed_precision = num_params_transformer_model * (2 + 4)
mem_grads_mixed_precision = num_params_transformer_model * (2 + 4)
mem_adam_states = num_params_transformer_model * (4 + 4)
gb = 1024**3
gb_mem_train_state = (
    sum([mem_params_mixed_precision, mem_grads_mixed_precision, mem_adam_states]) / gb
)
print(f"gb_mem_train_state = {gb_mem_train_state:.2f} GB")

gb_mem_train_state = 1205.04 GB


In [12]:
mem_zero3_train_per_gpu = (
    sum([mem_params_mixed_precision, mem_grads_mixed_precision, mem_adam_states])
    / num_gpus
)

mem_zero3_train_per_gpu_gb = mem_zero3_train_per_gpu / gb
print(f"mem_zero3_train_per_gpu_gb = {mem_zero3_train_per_gpu_gb:.2f} GB")

mem_zero3_train_per_gpu_gb = 301.26 GB


In [13]:
mem_activation_per_transformer_layer_per_gpu = 34 * b * s * h + 5 * b * s**2 * a
mem_activation_transformer_model_per_gpu = (
    num_hidden_layers * mem_activation_per_transformer_layer_per_gpu
)
gb_mem_activation_transformer_model_per_gpu = (
    mem_activation_transformer_model_per_gpu / gb
)
print(
    f"gb_mem_activation_transformer_model_per_gpu = {gb_mem_activation_transformer_model_per_gpu:.2f} GB"
)

gb_mem_activation_transformer_model_per_gpu = 92.50 GB


In [14]:
mem_kv_cache_per_token = b * h * l * 2 * 2
mem_kv_cache = mem_kv_cache_per_token * s
gb_men_kv_cache = mem_kv_cache / gb
print(f"gb_men_kv_cache = {gb_men_kv_cache:.2f} GB")

gb_men_kv_cache = 5.00 GB
